# 🔥 VTrackAI Studio - Kaggle E2E GPU Setup

**Prerequisites:**
1. Settings → Accelerator → **GPU P100** or **T4 x2**
2. Add-ons → Secrets → Add `HF_TOKEN` (from huggingface.co/settings/tokens)
3. Enable **Internet** in the sidebar
4. Run all cells (Ctrl+Shift+Enter)

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 1: SETUP (Run once per session, ~3-5 min)
# ═══════════════════════════════════════════════════════════════════════════════
import torch
import os
import sys
import subprocess

BASE_DIR = "/kaggle/working"
os.chdir(BASE_DIR)

# GPU Check
print("="*60)
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"📊 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ NO GPU! Go to Settings → Accelerator → GPU P100 or T4")
    sys.exit(1)
print("="*60)

# Clean start
subprocess.run(["rm", "-rf", f"{BASE_DIR}/vtrack-ai-studio"])

# Install dependencies
print("\n📦 Installing dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "torch", "torchvision", "--index-url", "https://download.pytorch.org/whl/cu121"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "fastapi", "uvicorn[standard]", "python-multipart", "huggingface_hub", "requests", "decord"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "numpy==1.26.4"])

# Node.js 18 via nodeenv
print("\n📦 Installing Node.js 18...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "nodeenv"])
subprocess.run(["nodeenv", "--node=18.20.0", f"{BASE_DIR}/node_env", "--force", "-q"])

NODE_BIN = f"{BASE_DIR}/node_env/bin"
os.environ["PATH"] = f"{NODE_BIN}:{os.environ['PATH']}"

subprocess.run([f"{NODE_BIN}/node", "--version"])
subprocess.run([f"{NODE_BIN}/npm", "--version"])

# Install localtunnel
print("\n📦 Installing localtunnel...")
subprocess.run([f"{NODE_BIN}/npm", "install", "-g", "localtunnel", "--silent"])

# Clone VTrackAI
print("\n📦 Cloning VTrackAI...")
subprocess.run(["git", "clone", "-q", "https://github.com/Amitro123/vtrack-ai-studio", f"{BASE_DIR}/vtrack-ai-studio"])
subprocess.run([sys.executable, "-m", "pip", "install", "-r", f"{BASE_DIR}/vtrack-ai-studio/backend/requirements.txt", "-q"], stderr=subprocess.DEVNULL)

# Clone SAM3 INTO backend/sam3 (where server.py expects it)
print("\n📦 Cloning SAM3 into backend/sam3...")
BACKEND_DIR = f"{BASE_DIR}/vtrack-ai-studio/backend"
os.chdir(BACKEND_DIR)
subprocess.run(["rm", "-rf", "./sam3"])
subprocess.run(["git", "clone", "-q", "https://github.com/facebookresearch/sam3", "./sam3"])
subprocess.run([sys.executable, "-m", "pip", "install", "-e", "./sam3", "-q"], stderr=subprocess.DEVNULL)
print("✅ SAM3 installed in backend/sam3")

# HuggingFace login + checkpoint
print("\n🔐 Downloading SAM3 checkpoint...")
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login, hf_hub_download

user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
login(token=HF_TOKEN, add_to_git_credential=False)

checkpoint_dir = f"{BACKEND_DIR}/checkpoints/sam3"
checkpoints_subdir = f"{checkpoint_dir}/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(checkpoints_subdir, exist_ok=True)

checkpoint_path = hf_hub_download(
    repo_id="facebook/sam3",
    filename="sam3.pt",
    local_dir=checkpoint_dir,
    token=HF_TOKEN,
)
print(f"✅ Downloaded: {checkpoint_path}")

# Create symlinks
sam3_pt = f"{checkpoint_dir}/sam3.pt"
symlinks = [
    f"{checkpoint_dir}/sam3_hiera_large.pt",
    f"{checkpoints_subdir}/sam3_hiera_tiny.pt",
    f"{checkpoints_subdir}/sam3_hiera_large.pt",
]

for dst in symlinks:
    try:
        if os.path.islink(dst) or os.path.exists(dst):
            os.remove(dst)
        os.symlink(sam3_pt, dst)
        print(f"✅ Symlink: {os.path.basename(dst)}")
    except OSError as e:
        print(f"❌ Symlink failed: {dst} - {e}")

os.chdir(BASE_DIR)

print("\n" + "="*60)
print("✅ SETUP COMPLETE! Run Cell 2 to start backend.")
print("="*60)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 2: START BACKEND + TUNNEL
# ═══════════════════════════════════════════════════════════════════════════════
import subprocess
import time
import requests
import os
import re
import urllib.request

BASE_DIR = "/kaggle/working"
BACKEND_DIR = f"{BASE_DIR}/vtrack-ai-studio/backend"
NODE_BIN = f"{BASE_DIR}/node_env/bin"
os.environ["PATH"] = f"{NODE_BIN}:{os.environ['PATH']}"

# Get public IP
print("🔍 Getting public IP...")
try:
    PUBLIC_IP = urllib.request.urlopen('https://api.ipify.org').read().decode('utf8')
    print(f"✅ Public IP: {PUBLIC_IP}")
except Exception:
    PUBLIC_IP = "(could not detect)"

# Kill existing
subprocess.run(["pkill", "-9", "-f", "uvicorn"], stderr=subprocess.DEVNULL)
subprocess.run(["pkill", "-9", "-f", "lt"], stderr=subprocess.DEVNULL)
time.sleep(2)

# Verify SAM3
print("\n🔍 Checking SAM3...")
print(f"   SAM3: {'✅' if os.path.exists(f'{BACKEND_DIR}/sam3') else '❌'} found")
print(f"   Checkpoint: {'✅' if os.path.exists(f'{BACKEND_DIR}/checkpoints/sam3/sam3.pt') else '❌'} found")

# Start backend
print("\n🚀 Starting backend...")
log_file = open(f"{BASE_DIR}/backend.log", "w")
proc = subprocess.Popen(
    ["uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=log_file,
    stderr=subprocess.STDOUT,
    cwd=BACKEND_DIR,
)

# Wait for backend
backend_ready = False
for i in range(30):
    if proc.poll() is not None:
        print("❌ Backend crashed!")
        with open(f"{BASE_DIR}/backend.log") as f:
            print(f.read()[-2000:])
        break
    try:
        r = requests.get("http://localhost:8000/api/health", timeout=3)
        health = r.json()
        print(f"✅ Backend running - SAM3: {'✅' if health.get('sam3_available') else '❌'}")
        backend_ready = True
        break
    except Exception:
        print(f"⏳ Waiting... ({i+1}/30)")
        time.sleep(2)

if backend_ready:
    # Start tunnel
    print("\n🌐 Creating backend tunnel...")
    tunnel_log = open(f"{BASE_DIR}/backend_tunnel.log", "w")
    subprocess.Popen([f"{NODE_BIN}/lt", "--port", "8000"], stdout=tunnel_log, stderr=subprocess.STDOUT)
    time.sleep(8)
    
    with open(f"{BASE_DIR}/backend_tunnel.log") as f:
        urls = re.findall(r'https://[\w-]+\.loca\.lt', f.read())
    
    if urls:
        with open(f"{BASE_DIR}/backend_url.txt", "w") as f:
            f.write(urls[0])
        print(f"\n" + "="*60)
        print(f"✅ Backend: {urls[0]}")
        print(f"🔑 Password: {PUBLIC_IP}")
        print("="*60)
        print("\n✅ Run Cell 3 to warm up SAM3.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 3: SAM3 WARM-UP (Pre-compute cache)
# ═══════════════════════════════════════════════════════════════════════════════
import requests
import time

print("🔥 Warming up SAM3 model...")
print("   This may take 30-60 seconds on first run.")

try:
    # Call warmup endpoint
    r = requests.post("http://localhost:8000/api/warmup", timeout=120)
    result = r.json()
    print(f"\n✅ Warm-up result: {result}")
except Exception as e:
    print(f"❌ Warm-up failed: {e}")

# Final health check
print("\n📊 Final health check:")
health = requests.get("http://localhost:8000/api/health").json()
print(f"   SAM3 Available: {health.get('sam3_available')}")
print(f"   SAM3 Initialized: {health.get('sam3_initialized')}")
print(f"   Device: {health.get('device')}")

print("\n" + "="*60)
print("✅ SAM3 is ready! Run Cell 4 to start frontend.")
print("="*60)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 4: START FRONTEND + TUNNEL
# ═══════════════════════════════════════════════════════════════════════════════
import subprocess
import time
import os
import re
import urllib.request

BASE_DIR = "/kaggle/working"
NODE_BIN = f"{BASE_DIR}/node_env/bin"
os.environ["PATH"] = f"{NODE_BIN}:{os.environ['PATH']}"

try:
    PUBLIC_IP = urllib.request.urlopen('https://api.ipify.org').read().decode('utf8')
except Exception:
    PUBLIC_IP = "(could not detect)"

# Read backend URL
try:
    with open(f"{BASE_DIR}/backend_url.txt") as f:
        BACKEND_URL = f.read().strip()
    print(f"📡 Backend: {BACKEND_URL}")
except Exception:
    BACKEND_URL = "http://localhost:8000"
    print("⚠️ Using localhost backend")

# Kill existing frontend
subprocess.run(["pkill", "-9", "-f", "vite"], stderr=subprocess.DEVNULL)
time.sleep(1)

# Install frontend
print("\n🚀 Installing frontend...")
os.chdir(f"{BASE_DIR}/vtrack-ai-studio")

with open(".env", "w") as f:
    f.write(f"VITE_API_URL={BACKEND_URL}\n")

subprocess.run([f"{NODE_BIN}/npm", "install", "--silent"], stderr=subprocess.DEVNULL)

# Start Vite
print("🚀 Starting frontend...")
vite_log = open(f"{BASE_DIR}/vite.log", "w")
subprocess.Popen(
    [f"{NODE_BIN}/npm", "run", "dev", "--", "--host", "0.0.0.0", "--port", "4173"],
    stdout=vite_log, stderr=subprocess.STDOUT,
    cwd=f"{BASE_DIR}/vtrack-ai-studio"
)
time.sleep(15)

# Start tunnel
print("\n🌐 Creating frontend tunnel...")
tunnel_log = open(f"{BASE_DIR}/frontend_tunnel.log", "w")
subprocess.Popen([f"{NODE_BIN}/lt", "--port", "4173"], stdout=tunnel_log, stderr=subprocess.STDOUT)
time.sleep(8)

with open(f"{BASE_DIR}/frontend_tunnel.log") as f:
    urls = re.findall(r'https://[\w-]+\.loca\.lt', f.read())

if urls:
    print("\n" + "="*60)
    print("🎉 VTrackAI Studio is LIVE!")
    print(f"\n   👉 FRONTEND: {urls[0]}")
    print(f"   📡 BACKEND:  {BACKEND_URL}")
    print(f"\n   🔑 PASSWORD: {PUBLIC_IP}")
    print("\n   ⚠️ Enter password on BOTH URLs!")
    print("="*60)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 5: GET URLS (Run if needed)
# ═══════════════════════════════════════════════════════════════════════════════
import re
import urllib.request

BASE_DIR = "/kaggle/working"
try:
    PUBLIC_IP = urllib.request.urlopen('https://api.ipify.org').read().decode('utf8')
except:
    PUBLIC_IP = "unknown"

print("=" * 60)
try:
    with open(f"{BASE_DIR}/backend_tunnel.log") as f:
        urls = re.findall(r'https://[\w-]+\.loca\.lt', f.read())
        if urls: print(f"📡 BACKEND: {urls[0]}")
except: pass

try:
    with open(f"{BASE_DIR}/frontend_tunnel.log") as f:
        urls = re.findall(r'https://[\w-]+\.loca\.lt', f.read())
        if urls: print(f"👉 FRONTEND: {urls[0]}")
except: pass

print(f"\n🔑 PASSWORD: {PUBLIC_IP}")
print("=" * 60)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 6: DEBUG LOGS
# ═══════════════════════════════════════════════════════════════════════════════
import os
import subprocess
BASE_DIR = "/kaggle/working"

print("=== SAM3 Check ===")
sam3_path = f"{BASE_DIR}/vtrack-ai-studio/backend/sam3"
print(f"Exists: {os.path.exists(sam3_path)}")

print("\n=== Checkpoints ===")
ckpt_dir = f"{BASE_DIR}/vtrack-ai-studio/backend/checkpoints/sam3"
if os.path.exists(ckpt_dir):
    for f in os.listdir(ckpt_dir):
        path = os.path.join(ckpt_dir, f)
        if os.path.islink(path):
            print(f"  {f} -> symlink")
        elif os.path.isfile(path):
            print(f"  {f} ({os.path.getsize(path)/1e6:.1f}MB)")

print("\n=== Backend Log (last 30 lines) ===")
try:
    with open(f"{BASE_DIR}/backend.log") as f:
        print("".join(f.readlines()[-30:]))
except: print("No log")

print("\n=== Processes ===")
result = subprocess.run(["ps", "aux"], capture_output=True, text=True)
for line in result.stdout.split("\n"):
    if any(x in line for x in ["uvicorn", "node", "lt"]) and "grep" not in line:
        print(line)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 7: CLEANUP
# ═══════════════════════════════════════════════════════════════════════════════
import subprocess
print("🧹 Cleaning up...")
subprocess.run(["pkill", "-9", "-f", "uvicorn"], stderr=subprocess.DEVNULL)
subprocess.run(["pkill", "-9", "-f", "node"], stderr=subprocess.DEVNULL)
subprocess.run(["pkill", "-9", "-f", "lt"], stderr=subprocess.DEVNULL)
print("✅ Done")